RFM analysis is a technique used to segment customers based on several factors.
Cliets who score high are expected to better client - Ie. Purchase more often, purchase higher amounts, or have purchase recently.
We'd like to know who these clients are so we can better direct our marketing efforts.

Clients are grouped based on three categories.
1. Recency (R) - How recent they placed an order
2. Frequency (F) - How often they place orders
3. Monetary (M) - How much the customer spends

Data for RFM analysis typically comes from the company's CRM system as is generally easily accessible.
In our example, we've writen a simple sql query to return prior invoices. We'll then use these in our analysis.


First, we'll import our packages and data. Note that the data's Client ID and Name fields have been altered.

In [43]:
import collections
import pandas as pd
import numpy  as np
from datetime import datetime
from matplotlib import pyplot as plt


data = pd.read_csv('sql_anon_invoices.csv')
data['invoice_date'] = data['invoice_date'].astype('datetime64[ns]')
data.head()

,client_id,client_description,invoice_number,amount,invoice_date
0,330368388,E.I. Du Pont de Nemours and Company,36767,344.06,1998-02-04
1,239904462,Cisco Systems Inc.,38222,109.02,1998-07-06
2,330368388,E.I. Du Pont de Nemours and Company,38252,145.00,1998-07-16
3,414495096,"Dole Food Company, Inc.",41782,1584.21,1999-08-04
4,605184616,On Semiconductor Corp,42018,85.00,1999-10-12


In [45]:
print("Rows: {}".format(len(data)))

Rows:55 12007


As we can see, we have approximately 12,000 invoices.
For our analysis, we need to group these invoices by client.

In [32]:
#create consolidated df
rfm = data[['client_id', 'client_description']].drop_duplicates()

Now we need to get the count of invoices for each client.

In [33]:
c_invoices_dict = collections.Counter(data['client_id'])

#count number of invoices for client (Frequency)
def get_counts(x):

    return c_invoices_dict[x.client_id]

rfm['F_count_invoices'] = rfm.apply(get_counts, axis=1)
rfm.head()


,client_id,client_description,F_count_invoices
0,330368388,E.I. Du Pont de Nemours and Company,3
1,239904462,Cisco Systems Inc.,1
3,414495096,"Dole Food Company, Inc.",1
4,605184616,On Semiconductor Corp,3
5,306505041,Costco Wholesale Corp.,1


Now we'll get the length of time they've been a client (expressed in years).

In [34]:
#get length of client history (date_diff between first and last invoice)
def get_history(x):

    d = data[data['client_id'] == x.client_id]
    d_max = max(d['invoice_date'])
    t = datetime.today()
    y = t - d_max

    return y.days

rfm['R_days_since_last_invoice'] = rfm.apply(get_history,axis=1)
rfm.head()

,client_id,client_description,F_count_invoices,R_days_since_last_invoice
0,330368388,E.I. Du Pont de Nemours and Company,3,5523
1,239904462,Cisco Systems Inc.,1,8114
3,414495096,"Dole Food Company, Inc.",1,7720
4,605184616,On Semiconductor Corp,3,6629
5,306505041,Costco Wholesale Corp.,1,7538


Now we need to find the total amount of work we've billed each client.

In [35]:
def total_invoices(x):
    d = data[data['client_id'] == x.client_id]
    total = d.amount.sum()
    return total

rfm['M_total_invoiced'] = rfm.apply(total_invoices, axis=1)
rfm.head()

#Remove everyone who hasn't had in invoice in more than 2 years.
#############################
rfm = rfm[rfm['R_days_since_last_invoice'] < 365 * 2]
############################

Finally, we need to determine where our cutoffs for each group are.
We'll split each category into 4 categories.


In [36]:
def get_ranks(x):

   pct_75 = np.percentile(x, 75)
   pct_50 = np.percentile(x, 50)
   pct_25 = np.percentile(x, 25)

   return {'max':x.max(), '75':pct_75, '50':pct_50, '25':pct_25, 'min':x.min()}


R_pct = get_ranks(rfm.R_days_since_last_invoice)
F_pct = get_ranks(rfm.F_count_invoices)
M_pct = get_ranks(rfm.R_days_since_last_invoice)

print(R_pct)
print(F_pct)
print(M_pct)

{'max': 721, '75': 280.0, '50': 190.0, '25': 145.0, 'min': 19}
{'max': 127, '75': 10.0, '50': 9.0, '25': 4.0, 'min': 1}
{'max': 721, '75': 280.0, '50': 190.0, '25': 145.0, 'min': 19}


In [37]:
def rank_r(x):
    i = x.R_days_since_last_invoice
    r = 0
    if i <= R_pct['25']:
        r = 1
    if (i >= R_pct['25']) and (i < R_pct['50']):
        r = 2
    if (i >= R_pct['50']) and (i < R_pct['75']):
        r = 3
    if i >= R_pct['75']:
        r = 4
    return r

rfm['R'] = rfm.apply(rank_r, axis=1)

def rank_f(x):
    i = x.F_count_invoices
    r = 0
    if i <= F_pct['25']:
        r = 4
    if (i >= F_pct['25']) and (i < F_pct['50']):
        r = 3
    if (i >= F_pct['50']) and (i < F_pct['75']):
        r = 2
    if i >= F_pct['75']:
        r = 1
    return r

rfm['F'] = rfm.apply(rank_f, axis=1)

def rank_m(x):
    i = x.M_total_invoiced
    r = 0
    if i <= M_pct['25']:
        r = 4
    if (i >= M_pct['25']) and (i < M_pct['50']):
        r = 3
    if (i >= M_pct['50']) and (i < M_pct['75']):
        r = 2
    if i >= M_pct['75']:
        r = 1
    return r


rfm['M'] = rfm.apply(rank_m, axis=1)
rfm.head()

,client_id,client_description,F_count_invoices,R_days_since_last_invoice,M_total_invoiced,R,F,M
8,291051913,CMS Energy Corp.,127,35,51935.19,1,1,1
11,324609387,"DST Systems, Inc.",31,28,45421.39,1,1,1
141,109121390,"D&K Healthcare Resources, Inc.",78,70,16555.00,1,1,1
142,114967358,Equity Residential Properties Trust,36,41,37725.00,1,1,1
152,45005285,Smith International Inc,15,182,21591.48,2,1,1


Now time to calculate who are our best clients (all 1's). We should be in constant contact with these clients to make them feel valued & appreciated. These customers represent a disproportionately large portion of revenues.

In [38]:
best_clients = rfm[(rfm['R'] ==1) & (rfm['F'] == 1) & (rfm['M']==1)]
best_clients


,client_id,client_description,F_count_invoices,R_days_since_last_invoice,M_total_invoiced,R,F,M
8,291051913,CMS Energy Corp.,127,35,51935.19,1,1,1
11,324609387,"DST Systems, Inc.",31,28,45421.39,1,1,1
141,109121390,"D&K Healthcare Resources, Inc.",78,70,16555.00,1,1,1
142,114967358,Equity Residential Properties Trust,36,41,37725.00,1,1,1
272,555339793,Coventry Health Care Inc.,42,103,30840.00,1,1,1
...,...,...,...,...,...,...,...,...
8514,196016076,"The Bank of New York Company, Inc.",29,99,22711.00,1,1,1
9436,100365765,Equitable Resources Inc.,12,19,7995.00,1,1,1
9555,106944318,"American International Group, Inc.",14,56,3235.00,1,1,1
10080,406721556,Baker Hughes Incorporated,20,25,10710.00,1,1,1


High-Spending New Customers (1-4-1 & 1-4-2).

In [39]:
high_spending_clients = rfm[(rfm['R'] ==1) & (rfm['F'] == 4) & ((rfm['M']==1) | (rfm['M']==2))]
high_spending_clients

,client_id,client_description,F_count_invoices,R_days_since_last_invoice,M_total_invoiced,R,F,M
6587,172038105,Paloma Fletcher,3,75,545.00,1,4,1
8082,657441193,Aria Davenport,3,118,975.00,1,4,1
8837,227166578,Tyson Rasmussen,3,92,1475.00,1,4,1
10067,411867077,Flowserv Corp,2,76,2980.00,1,4,1
10372,491845922,UGI Corporation,2,109,1845.00,1,4,1
10579,615504366,W.W. Grainger Inc,2,112,2140.00,1,4,1
10701,99560365,American Power Conversion Corporation,2,49,1390.00,1,4,1
10836,530941112,Crompton Corp.,3,41,15250.00,1,4,1
10889,323284806,Western Digital Inc,2,91,350.00,1,4,1
11439,113959053,Cash Frederick,3,62,415.00,1,4,1


Lowest-Spending Active Loyal Customers (1-1-3 & 1-1-4)

In [40]:
low_spending_active_loyal_clients = rfm[((rfm['R'] ==1) & (rfm['F'] == 1) & ((rfm['M']==3) | (rfm['M']==4)))]
low_spending_active_loyal_clients

,client_id,client_description,F_count_invoices,R_days_since_last_invoice,M_total_invoiced,R,F,M


Churned Best Customers (4-1-1) (4-1-2) (4-2-2). These were the good customers that we let "get away". We should consider reconnecting with them.

In [41]:

churned_best_customers = rfm[((rfm['R'] ==4) & (rfm['F'] == 1) & ((rfm['M']==1) | (rfm['M']==2))) |
                             ((rfm['R'] ==4) & (rfm['F'] == 2) & (rfm['M']==2))
                             ]
churned_best_customers

,client_id,client_description,F_count_invoices,R_days_since_last_invoice,M_total_invoiced,R,F,M
297,548351086,"Anheuser-Busch Companies, Inc.",11,487,18035.48,4,1,1
304,301703542,H.J. Heinz Company,16,290,20820.00,4,1,1
308,73615687,Pulte Homes Inc,10,349,1280.00,4,1,1
337,52988469,"Arrow Electronics, Inc.",44,677,10065.00,4,1,1
412,157893822,Pride International Inc,10,280,5610.00,4,1,1
414,637626070,Exelon Corporation,40,483,76938.00,4,1,1
419,680400945,Mattel Inc.,13,280,4080.00,4,1,1
424,327519152,Bank One Corporation,104,280,21540.00,4,1,1
443,17847412,Citizens Communications Co.,10,280,31495.00,4,1,1
446,54488403,"American Water Works Company, Inc.",17,469,6155.00,4,1,1


Finally, let's visualize our customers. 

In [42]:
import plotly.express as px

fig = px.scatter_3d(rfm, x='R_days_since_last_invoice', y='F_count_invoices', z='M_total_invoiced', hover_name='client_description', color='M_total_invoiced')
fig.update_layout(scene = dict(
                    xaxis_title='Days Since Last Invoice',
                    yaxis_title='Count Total Invoices',
                    zaxis_title='Total Amount Invoiced'))

fig.show()


